In [ ]:
import pandas as pd
from pathlib import Path
import yaml
from definitions import *

DATASET_PATH = "../tpmscan-dataset"

def load_support(dir):
    rows = []
    for result in Path(dir).rglob("results.yaml"):
        with open(result, "r") as f:
            data = yaml.safe_load(f)

        support = {}
        support["device"] = data['Manufacturer'] + " " + data['Firmware version']
        support["year"] = data['Capability_properties-fixed']["TPM2_PT_YEAR"]["raw"]
        support["day"] = data['Capability_properties-fixed']["TPM2_PT_DAY_OF_YEAR"]["raw"]
        support["revision"] = data['Capability_properties-fixed']["TPM2_PT_REVISION"]["value"]

        for alg_id, alg_name in TPM_ALG_IDS.items():
            support[alg_name] = alg_id in data["Capability_algorithms"]

        unknown_algs = set(data["Capability_algorithms"]) - set(TPM_ALG_IDS.keys())
        if unknown_algs:
            print(f"Unknown alg IDs: {unknown_algs} in {result}")

        if "Capability_ecc-curves" not in data or data["Capability_ecc-curves"] is None:
            data["Capability_ecc-curves"] = []

        for _, ecc_name in TPM_ECC_IDS.items():
            support[ecc_name] = ecc_name in data["Capability_ecc-curves"]

        unknown_eccs = set(data["Capability_ecc-curves"]) - set(TPM_ECC_IDS.values())
        if unknown_eccs:
            print(f"Unknown ECCs: {unknown_eccs} in {result}")

        for cc_id, cc_name in TPM_CC_IDS.items():
            support[cc_name] = cc_id in data["Capability_commands"]

        unknown_ccs = set(data["Capability_commands"]) - set(TPM_CC_IDS.keys())
        if unknown_ccs:
            print(f"Unknown CCs: {unknown_ccs} in {result}")

        rows.append(support)

    return pd.DataFrame(rows).fillna(False)

support_df = load_support(DATASET_PATH)

support_df = support_df.groupby("device").agg(lambda x: x.iloc[0] if x.nunique() == 1 else pd.NA).reset_index()
print(len(support_df), "firmware loaded")


In [ ]:
alg_columns = [
    "device",
    "year",
    "day",
    "revision",
    "TPM2_ALG_ECC",
    "TPM2_ALG_ECDH",
    "TPM2_ALG_ECDSA",
    "TPM2_ALG_ECDAA",
    "TPM2_ALG_ECSCHNORR",
    "TPM2_ALG_ECMQV",
    "TPM2_ALG_SM2",
    "TPM2_ALG_RSA",
    "TPM2_ALG_RSASSA",
    "TPM2_ALG_RSAES",
    "TPM2_ALG_RSAPSS",
    "TPM2_ALG_OAEP",
    "TPM2_ALG_TDES",
    "TPM2_ALG_AES",
    "TPM2_ALG_SYMCIPHER",
    "TPM2_ALG_CAMELLIA",
    "TPM2_ALG_SM4",
    "TPM2_ALG_SHA",
    "TPM2_ALG_SHA256",
    "TPM2_ALG_SHA384",
    "TPM2_ALG_SHA512",
    "TPM2_ALG_SHA3_256",
    "TPM2_ALG_SHA3_384",
    "TPM2_ALG_SHA3_512",
    "TPM2_ALG_SM3_256",
    "TPM2_ALG_ECB",
    "TPM2_ALG_CBC",
    "TPM2_ALG_CTR",
    "TPM2_ALG_OFB",
    "TPM2_ALG_CFB",
    "TPM2_ALG_NULL",
    "TPM2_ALG_XOR",
    "TPM2_ALG_HMAC",
    "TPM2_ALG_CMAC",
    "TPM2_ALG_MGF1",
    "TPM2_ALG_KEYEDHASH",
    "TPM2_ALG_KDF1_SP800_56A",
    "TPM2_ALG_KDF1_SP800_108",
    "TPM2_ALG_KDF2",
]

alg_df = pd.DataFrame(support_df, columns=alg_columns)
alg_df[alg_columns[4:]].sum() / alg_df.shape[0] * 100


In [ ]:
# TPM2_ECC support table
ecc_columns = [
    "device",
    "year",
    "day",
    "revision",
    "TPM2_ECC_NONE",
    "TPM2_ECC_NIST_P192",
    "TPM2_ECC_NIST_P224",
    "TPM2_ECC_NIST_P256",
    "TPM2_ECC_NIST_P384",
    "TPM2_ECC_NIST_P521",
    "TPM2_ECC_BN_P256",
    "TPM2_ECC_BN_P683",
    "TPM2_ECC_SM2_P256",
]

ecc_df = pd.DataFrame(support_df, columns=ecc_columns)
ecc_df[ecc_columns[4:]].sum() / ecc_df.shape[0] * 100


In [ ]:
# TPM2_CC support table
cc_columns = [
    "device",
    "year",
    "day",
    "revision",
    "TPM2_CC_RSA_Encrypt",
    "TPM2_CC_RSA_Decrypt",
    "TPM2_CC_ECDH_KeyGen",
    "TPM2_CC_ECDH_ZGen",
    "TPM2_CC_ZGen_2Phase",
    "TPM2_CC_Commit",
    "TPM2_CC_EC_Ephemeral",
    "TPM2_CC_Sign",
]

cc_df = pd.DataFrame(support_df, columns=cc_columns)
cc_df[cc_columns[4:]].sum() / cc_df.shape[0] * 100
